In [1]:
import numpy as np
import os
import xml.etree.ElementTree as ET
import tensorflow as tf
import keras
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from sklearn.model_selection import train_test_split
import random

In [2]:
# CHANGE THIS to the path on your machine
projectDir = r'C:\Users\Viktor\Skola\embedded\embedded_project'

In [3]:
homeDir = os.path.join(projectDir,'code')
xmlDir = os.path.join(projectDir, 'Stanford40_XMLAnnotations\XMLAnnotations')

In [4]:
x = np.load(os.path.join(homeDir,'partCoords.npy'))
print(x.shape)

(9532, 18, 2)


In [7]:
#np.set_printoptions(suppress=True)

#x10 = x[:10]

#print(x10)

#x10_norm = np.zeros(x10.shape)

for i in range(len(x)):
    #print(x10[i])
    xmax, ymax = np.nanmax(x[i], axis=0)
    xmin, ymin = np.nanmin(x[i], axis=0)
    
    #if np.isnan(xmax):
    #    print(x[i])
    #print(i)
    #print(xmin)
    #print(xmax)
    #print(ymin)
    #print(ymax)
    
    x[i][:,0] -= xmin
    x[i][:,0] /= (xmax - xmin)
    
    x[i][:,1] -= ymin
    x[i][:,1] /= (ymax - ymin)
    
    #print(x[i])
#print(x10_norm)
#print(x[1297])

<ipython-input-7-733908754da7>:11: RuntimeWarning: All-NaN slice encountered
  xmax, ymax = np.nanmax(x[i], axis=0)
<ipython-input-7-733908754da7>:12: RuntimeWarning: All-NaN slice encountered
  xmin, ymin = np.nanmin(x[i], axis=0)


In [8]:
x[np.isnan(x)] = 0

In [9]:
print(x)

[[[0.54545455 0.04210526]
  [0.4171123  0.29824561]
  [0.07486631 0.34035088]
  ...
  [0.55080214 0.        ]
  [0.25668449 0.08070175]
  [0.         0.        ]]

 [[0.46       0.05291005]
  [0.445      0.47619048]
  [0.1        0.49206349]
  ...
  [0.525      0.00529101]
  [0.265      0.1005291 ]
  [0.6        0.11640212]]

 [[0.39344262 0.11111111]
  [0.78688525 0.51111111]
  [1.         0.43333333]
  ...
  [0.45901639 0.01111111]
  [0.         0.        ]
  [0.74590164 0.        ]]

 ...

 [[0.78723404 0.29166667]
  [0.56382979 0.48611111]
  [0.24468085 0.40277778]
  ...
  [0.84574468 0.22916667]
  [0.57978723 0.        ]
  [0.         0.        ]]

 [[0.53784861 0.18315018]
  [0.60756972 0.27838828]
  [0.35258964 0.23076923]
  ...
  [0.62749004 0.10989011]
  [0.46414343 0.        ]
  [0.80278884 0.        ]]

 [[0.24418605 0.2755102 ]
  [0.70930233 0.34693878]
  [0.38372093 0.32653061]
  ...
  [0.34883721 0.15306122]
  [0.         0.        ]
  [0.80232558 0.        ]]]


In [10]:
print(x[0].shape)

(18, 2)


In [11]:
os.chdir(homeDir)

jpgOrder = open('JPG_Order.txt', 'r')
lines = jpgOrder.readlines()

y_string = []

os.chdir(xmlDir)

for i in range(len(lines)):
    imgFileName = lines[i].strip()
    fileName = imgFileName.split(".")[0]
    xmlFileName = fileName + ".xml"
    
    root = ET.parse(xmlFileName).getroot()
    action = root.find('object/action').text
    
    y_string.append(action)
    
    print(str(i) + '/' + str(len(lines)),end='\r')

#print(y_string)

In [12]:
(allUniqueClasses,y_int) = np.unique(y_string, return_inverse=True)

print(y_int)

y_cat = tf.keras.utils.to_categorical(y_int, len(allUniqueClasses))

print(y_cat)

[ 0  0  0 ... 39 39 39]
[[1. 0. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 1.]
 [0. 0. 0. ... 0. 0. 1.]
 [0. 0. 0. ... 0. 0. 1.]]


In [13]:
x_train, x_test, y_train, y_test = train_test_split(x, y_cat, test_size=0.2, random_state=42)

In [39]:
def defineModel(nrOfClasses):
    model = Sequential()
    model.add(keras.Input(shape=(18,2)))
    model.add(Flatten())
    model.add(Dense(128, activation='relu', kernel_initializer='he_uniform'))
    model.add(Dense(256, activation='relu', kernel_initializer='he_uniform'))
    model.add(Dense(128, activation='relu', kernel_initializer='he_uniform'))
    model.add(Dense(128, activation='relu', kernel_initializer='he_uniform'))
    
    model.add(Dense(nrOfClasses, activation='softmax'))
    
    opt = keras.optimizers.Adam(learning_rate=0.001)
    model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])
    model.summary()
    return model

In [40]:
model = defineModel(40)

Model: "sequential_228"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_228 (Flatten)        (None, 36)                0         
_________________________________________________________________
dense_1077 (Dense)           (None, 128)               4736      
_________________________________________________________________
dense_1078 (Dense)           (None, 256)               33024     
_________________________________________________________________
dense_1079 (Dense)           (None, 128)               32896     
_________________________________________________________________
dense_1080 (Dense)           (None, 128)               16512     
_________________________________________________________________
dense_1081 (Dense)           (None, 40)                5160      
Total params: 92,328
Trainable params: 92,328
Non-trainable params: 0
________________________________________________

In [41]:
model.fit(x_train,y_train,batch_size = 32,epochs=20,verbose=1)

Epoch 1/20
239/239 [==============================] - 0s 1ms/step - loss: 3.2622 - accuracy: 0.1289
Epoch 2/20
239/239 [==============================] - 0s 2ms/step - loss: 2.8291 - accuracy: 0.2250
Epoch 3/20
239/239 [==============================] - 0s 1ms/step - loss: 2.6327 - accuracy: 0.2609
Epoch 4/20
239/239 [==============================] - 0s 1ms/step - loss: 2.5009 - accuracy: 0.2947
Epoch 5/20
239/239 [==============================] - 0s 1ms/step - loss: 2.3980 - accuracy: 0.3214
Epoch 6/20
239/239 [==============================] - 0s 1ms/step - loss: 2.3053 - accuracy: 0.3397
Epoch 7/20
239/239 [==============================] - 0s 1ms/step - loss: 2.2199 - accuracy: 0.3618
Epoch 8/20
239/239 [==============================] - 0s 1ms/step - loss: 2.1451 - accuracy: 0.3727
Epoch 9/20
239/239 [==============================] - 0s 1ms/step - loss: 2.0799 - accuracy: 0.3995
Epoch 10/20
239/239 [==============================] - 0s 1ms/step - loss: 2.0015 - accuracy: 0.4101

In [42]:
model.evaluate(x_test,y_test)

60/60 [==============================] - 0s 783us/step - loss: 2.7600 - accuracy: 0.3152


[2.7599880695343018, 0.31515470147132874]

In [61]:
os.chdir(homeDir)
savedModelDir = os.path.join(homeDir, "saved_model")
if os.path.exists(savedModelDir):
    print("it exists")
else:
    print("does not exist, create folder")
    os.mkdir(savedModelDir)

model.save('saved_model/parts_model')

it exist
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
INFO:tensorflow:Assets written to: saved_model/parts_model\assets


In [62]:
#convert to tensorflow lite model
# Convert the model
converter = tf.lite.TFLiteConverter.from_saved_model(savedModelDir+'/parts_model') # path to the SavedModel directory
tflite_model = converter.convert()

os.makedirs(os.path.join(os.getcwd(),'saved_tflite_models'), exist_ok = True)
# Save the model.
with open('./saved_tflite_models/model_2.tflite', 'wb') as f:
    f.write(tflite_model)

In [67]:
#load tensorflow lite model
interpreter = tf.lite.Interpreter(model_path='./saved_tflite_models/model_2.tflite')
interpreter.allocate_tensors()

# Get input and output tensors.
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()


In [68]:
input_details

[{'name': 'input_229',
  'index': 0,
  'shape': array([ 1, 18,  2]),
  'shape_signature': array([-1, 18,  2]),
  'dtype': numpy.float32,
  'quantization': (0.0, 0),
  'quantization_parameters': {'scales': array([], dtype=float32),
   'zero_points': array([], dtype=int32),
   'quantized_dimension': 0},
  'sparsity_parameters': {}}]

In [83]:
x_test = x_test.astype('float32')

correct_predict = 0
for image_number in range(len(x_test)): 
    interpreter.set_tensor(input_details[0]['index'], x_test[image_number][np.newaxis, ...])
    interpreter.invoke()
    output_data = interpreter.get_tensor(output_details[0]['index'])
    if np.argmax(output_data[0]) == np.argmax(y_test[image_number]):
        correct_predict +=1

In [84]:
accuracy = correct_predict/len(y_test)
accuracy

0.31515469323544837

## Hyperparemeter tuning

In [30]:
import itertools

In [31]:
#General function that creates a Neural network model based on the input.
def createModel(denseHiddenLayer, outputLayer, inShape, nrOfclasses):
    
    model = Sequential()
    #input layer
    model.add(keras.Input(shape=inShape))
    model.add(Flatten())
    
    #hidden layers
    print(len(denseHiddenLayer))
    for layer in denseHiddenLayer:
        model.add(Dense(layer[0], activation=layer[1], kernel_initializer='he_uniform'))

    #outputlayer
    model.add(Dense(nrOfclasses, activation = outputLayer))
    
    #model.summary()
    return model

In [50]:

#User decide the input layer settings: use intergets only and not float 
#The numbers of neurons to test in the input layer

#user selects different hidden layers to try
nrHiddenLayers = [2,3,4,5]
#user selects the different neurons to try
nrHiddenLayersNeurons = [32,64,128,256,512]
#user selects the different activatons to try
hiddenLayersActivations = ['relu']
#combine every combination for hidden layers
nrHiddenLayersCombination = list(itertools.product(*[nrHiddenLayersNeurons, hiddenLayersActivations]))

#User select the output layers activation to try
outputLayerActivations = ['softmax']

#user selects SGD's learning rate (interget and float are valid)
learningRate = [0.01, 0.005, 0.001, 0.0005, 0.0001]

In [51]:
#This is an important function that creates every combinations for the NN model based on the selected data from the user.
def allModelData(nrhidLayers, nrhidLayersComb, outLayerAct, lRate ):
    
    hiddenLayerCombinations = []
    for i in nrhidLayers:
        temp = []
        for x in range(i):
            temp.append(nrhidLayersComb)
        hiddenLayerCombinations.append(temp)
    
    allHiddenLayerCombination = []
    for x in hiddenLayerCombinations:
        combb = list(itertools.product(*x))
        HiddenLayerCombination = list(itertools.product(*[combb, outLayerAct, lRate]))
        allHiddenLayerCombination.append(HiddenLayerCombination)
        
    print(len(allHiddenLayerCombination[0]))
    print(len(allHiddenLayerCombination[1]))

    
    all_model_combinations = list(itertools.chain.from_iterable(allHiddenLayerCombination))
    print(len(all_model_combinations))
    return all_model_combinations


all_model_combination = allModelData(nrHiddenLayers, nrHiddenLayersCombination, outputLayerActivations, learningRate )
#all_model_combination[:10]
print(len(all_model_combination))


125
625
19500
19500


In [52]:
print(all_model_combination[0][0])
print(all_model_combination[0][1])
print(all_model_combination[0][2])


((32, 'relu'), (32, 'relu'))
softmax
0.01


In [57]:
#list of the score and the model settings will be stored here 
results =[]
#Iterate through every combinations and appends the score and model settings in the list results.
count = 1
tries = 100
combination_indexes_to_try = random.sample(range(len(all_model_combination)),tries)
for attributes in [all_model_combination[i] for i in combination_indexes_to_try]:
    print(count,"of",tries)
    count += 1
    denseHiddenLayer_data = attributes[0]
    outputLayer_data = attributes[1]
    learningRate_data = attributes[2]
    print(denseHiddenLayer_data,outputLayer_data,learningRate_data)
    nn_model = createModel(denseHiddenLayer_data, outputLayer_data, (18,2), 40)
    opt = keras.optimizers.Adam(learning_rate=learningRate_data)
    #optimizer =keras.optimizers.SGD(learning_rate = learningRate_data)
    nn_model.compile(loss= keras.losses.categorical_crossentropy,
              optimizer =opt,
              metrics =['accuracy'] )
    nn_model.fit(x_train,y_train,
          batch_size = 32,
          epochs = 20, verbose=0)
    score = nn_model.evaluate(x_test,y_test, verbose = 0)
    print('Test loss:', score[0])
    print('Test accuracy:', score[1])
    results.append([(denseHiddenLayer_data, outputLayer_data, learningRate_data), score[1]])

1 of 100
((64, 'relu'), (512, 'relu'), (512, 'relu'), (256, 'relu'), (128, 'relu')) softmax 0.0001
5
Test loss: 2.5698211193084717
Test accuracy: 0.3020451068878174
2 of 100
((256, 'relu'), (256, 'relu'), (256, 'relu'), (256, 'relu'), (64, 'relu')) softmax 0.0005
5
Test loss: 2.680577516555786
Test accuracy: 0.33141058683395386
3 of 100
((128, 'relu'), (128, 'relu'), (512, 'relu'), (32, 'relu'), (512, 'relu')) softmax 0.005
5
Test loss: 3.673398733139038
Test accuracy: 0.02936549484729767
4 of 100
((128, 'relu'), (64, 'relu'), (128, 'relu'), (32, 'relu'), (64, 'relu')) softmax 0.005
5
Test loss: 2.661656141281128
Test accuracy: 0.30414262413978577
5 of 100
((512, 'relu'), (32, 'relu'), (64, 'relu'), (32, 'relu'), (512, 'relu')) softmax 0.0005
5
Test loss: 2.573690176010132
Test accuracy: 0.32197168469429016
6 of 100
((128, 'relu'), (512, 'relu'), (32, 'relu'), (128, 'relu'), (512, 'relu')) softmax 0.005
5
Test loss: 3.6733078956604004
Test accuracy: 0.03198741376399994
7 of 100
((512, 

Test loss: 2.56489634513855
Test accuracy: 0.30466702580451965
54 of 100
((128, 'relu'), (256, 'relu'), (512, 'relu'), (256, 'relu'), (256, 'relu')) softmax 0.0001
5
Test loss: 2.5040817260742188
Test accuracy: 0.32406923174858093
55 of 100
((512, 'relu'), (256, 'relu'), (256, 'relu'), (32, 'relu'), (512, 'relu')) softmax 0.01
5
Test loss: 3.673109769821167
Test accuracy: 0.029889879748225212
56 of 100
((32, 'relu'), (64, 'relu'), (64, 'relu'), (256, 'relu'), (32, 'relu')) softmax 0.0005
5
Test loss: 2.602445125579834
Test accuracy: 0.2852648198604584
57 of 100
((32, 'relu'), (128, 'relu'), (64, 'relu'), (512, 'relu'), (32, 'relu')) softmax 0.005
5
Test loss: 2.821166515350342
Test accuracy: 0.2585212290287018
58 of 100
((128, 'relu'), (256, 'relu'), (512, 'relu'), (64, 'relu')) softmax 0.001
4
Test loss: 3.0989227294921875
Test accuracy: 0.3025694787502289
59 of 100
((128, 'relu'), (128, 'relu'), (256, 'relu'), (32, 'relu'), (64, 'relu')) softmax 0.001
5
Test loss: 2.6634039878845215


In [58]:
#Show top 10 highest score (Accuracy) and the setting used.
def takeSecond(elem):
    return elem[1]
top10 = sorted(results, key= takeSecond, reverse = True)[:10]

In [59]:
top10

[[(((256, 'relu'), (256, 'relu'), (512, 'relu'), (64, 'relu'), (128, 'relu')),
   'softmax',
   0.0005),
  0.34084948897361755],
 [(((256, 'relu'), (512, 'relu'), (128, 'relu'), (64, 'relu'), (128, 'relu')),
   'softmax',
   0.0005),
  0.34032511711120605],
 [(((256, 'relu'), (64, 'relu'), (256, 'relu'), (128, 'relu'), (128, 'relu')),
   'softmax',
   0.0005),
  0.3377031981945038],
 [(((512, 'relu'), (256, 'relu'), (32, 'relu'), (128, 'relu')),
   'softmax',
   0.001),
  0.3366544246673584],
 [(((512, 'relu'), (512, 'relu'), (256, 'relu'), (32, 'relu')),
   'softmax',
   0.0005),
  0.3350812792778015],
 [(((512, 'relu'), (128, 'relu'), (256, 'relu'), (512, 'relu')),
   'softmax',
   0.0005),
  0.33298373222351074],
 [(((512, 'relu'), (256, 'relu'), (128, 'relu')), 'softmax', 0.001),
  0.33298373222351074],
 [(((256, 'relu'), (256, 'relu'), (256, 'relu'), (256, 'relu'), (64, 'relu')),
   'softmax',
   0.0005),
  0.33141058683395386],
 [(((512, 'relu'), (128, 'relu'), (32, 'relu'), (64,

In [ ]:
sortedAccuracy = sorted(results, key= takeSecond, reverse = True)

In [ ]:
np.save(homeDir+"/sortedAccuracy.npy", np.array(sortedAccuracy))

In [29]:
sortedAccuracy = np.load(os.path.join(homeDir,'sortedAccuracy.npy'),allow_pickle=True)
print(sortedAccuracy)

[[(((512, 'relu'), (512, 'relu'), (512, 'relu'), (512, 'relu')), 'softmax', 0.001)
  0.8815737962722778]
 [(((256, 'relu'), (512, 'relu'), (512, 'relu'), (512, 'relu')), 'softmax', 0.001)
  0.8771147727966309]
 [(((256, 'relu'), (256, 'relu'), (512, 'relu'), (512, 'relu')), 'softmax', 0.001)
  0.8737049102783203]
 [(((128, 'relu'), (512, 'relu'), (512, 'relu'), (512, 'relu')), 'softmax', 0.001)
  0.8710819482803345]
 [(((512, 'relu'), (256, 'relu'), (512, 'relu'), (512, 'relu')), 'softmax', 0.001)
  0.8641311526298523]
 [(((128, 'relu'), (256, 'relu'), (512, 'relu'), (512, 'relu')), 'softmax', 0.001)
  0.8625573515892029]
 [(((256, 'relu'), (512, 'relu'), (512, 'relu'), (256, 'relu')), 'softmax', 0.001)
  0.8574426174163818]
 [(((128, 'relu'), (512, 'relu'), (256, 'relu'), (512, 'relu')), 'softmax', 0.001)
  0.8553442358970642]
 [(((128, 'relu'), (512, 'relu'), (512, 'relu'), (256, 'relu')), 'softmax', 0.001)
  0.8532459139823914]
 [(((512, 'relu'), (512, 'relu'), (512, 'relu')), 'soft